# Try more complex promt

In [2]:
import pandas as pd
import anthropic

In [3]:
MODELS = dict(haiku="claude-3-haiku-20240307", sonnet="claude-3-5-sonnet-20240620")

In [4]:
df = pd.read_csv("data/train.csv", index_col=0)

In [5]:
client = anthropic.Anthropic()

In [21]:
def get_answer_2(task, model_name = MODELS['sonnet']): 
    system = """
You are tasked with solving a mathematical problem from the Russian EGE (Unified State Exam) exam. Your goal is to provide the correct numerical answer that can be used in a scoring script. Follow these instructions carefully:

1. Read the following mathematical task:
<task>
{{TASK}}
</task>

2. Solve the problem step by step. Use a logical approach and apply relevant mathematical concepts and formulas.

3. Show your work and explain your reasoning in detail. This will help verify the correctness of your solution.

4. After solving the problem, provide your final answer as a single number without any units or additional text.

5. Format your response as follows:
   <work>
   [Show your step-by-step solution and explanations here]
   </work>
   
   <answer>
   [Provide only the final numerical answer here]
   </answer>

Important: The <answer> section must contain only the final numerical result, without any additional text, units, or explanations. This is crucial for the scoring script to function correctly.

Remember to double-check your calculations and ensure that your final answer accurately reflects the solution to the given task.
"""
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=2000,
        system=system,
        messages=messages
    )
    return(response.content[0].text)

In [18]:
task, answer = df.iloc[4,]

In [22]:
quess = get_answer_2(task)

In [23]:
print(task)
print(f"Quess = {quess}")
print(f"Answer = {answer}")

A traveler from Moscow wants to visit four cities of the Golden Ring of Russia: Vladimir, Yaroslavl, Suzdal, and Rostov. The travel agency offers routes that include visits to some of the cities of the Golden Ring. Information about the cost of tickets and routes is presented in the table.

\begin{center}
\begin{tabular}{|c|c|c|c|}
\hline
Route number&Visited cities&Cost (rubles)\\\hline
1&Suzdal, Yaroslavl, Vladimir&3900 \\\hline
2&Rostov, Vladimir&2400 \\\hline
3&Yaroslavl, Vladimir&2100 \\\hline
4&Suzdal&1650 \\\hline
5&Rostov, Suzdal&2700 \\\hline
6&Yaroslavl, Rostov&2350 \\\hline
\end{tabular}
\end{center}

Which routes should the traveler choose to visit all four cities and spend less than 5000 rubles?

In the answer, specify any one set of route numbers without spaces, commas, or other additional symbols.
Quess = Let's approach this problem step by step:

<work>
1) First, we need to find combinations of routes that cover all four cities: Vladimir, Yaroslavl, Suzdal, and Rostov.


In [24]:
import re

def extract_answer_content(text):
    pattern = r'<answer>(.*?)</answer>'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return "No answer found"

In [28]:
def get_answer_2(task, model_name = MODELS['sonnet']): 
    system = """
You are tasked with solving a mathematical problem from the Russian EGE (Unified State Exam) exam. Your goal is to provide the correct numerical answer that can be used in a scoring script. Follow these instructions carefully:

1. Read the following mathematical task:
<task>
{{TASK}}
</task>

2. Solve the problem step by step. Use a logical approach and apply relevant mathematical concepts and formulas.

3. Show your work and explain your reasoning in detail. This will help verify the correctness of your solution.

4. After solving the problem, provide your final answer as a single number without any units or additional text.

5. Format your response as follows:
   <work>
   [Show your step-by-step solution and explanations here]
   </work>
   
   <answer>
   [Provide only the final numerical answer here]
   </answer>

Important: The <answer> section must contain only the final numerical result, without any additional text, units, or explanations. This is crucial for the scoring script to function correctly.

Remember to double-check your calculations and ensure that your final answer accurately reflects the solution to the given task.
"""
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=2000,
        system=system,
        temperature=0.1,
        messages=messages
    )
    return(extract_answer_content(response.content[0].text))

In [29]:
quess = get_answer_2(task)

In [30]:
print(task)
print(f"Quess = {quess}")
print(f"Answer = {answer}")

A traveler from Moscow wants to visit four cities of the Golden Ring of Russia: Vladimir, Yaroslavl, Suzdal, and Rostov. The travel agency offers routes that include visits to some of the cities of the Golden Ring. Information about the cost of tickets and routes is presented in the table.

\begin{center}
\begin{tabular}{|c|c|c|c|}
\hline
Route number&Visited cities&Cost (rubles)\\\hline
1&Suzdal, Yaroslavl, Vladimir&3900 \\\hline
2&Rostov, Vladimir&2400 \\\hline
3&Yaroslavl, Vladimir&2100 \\\hline
4&Suzdal&1650 \\\hline
5&Rostov, Suzdal&2700 \\\hline
6&Yaroslavl, Rostov&2350 \\\hline
\end{tabular}
\end{center}

Which routes should the traveler choose to visit all four cities and spend less than 5000 rubles?

In the answer, specify any one set of route numbers without spaces, commas, or other additional symbols.
Quess = 35
Answer = 53


# Try to calculate some more

In [32]:
from tqdm import tqdm

In [34]:
answers = []
quesses = []
for i in tqdm(range(10)):
    task, answer = df.iloc[i,]
    quess = get_answer_2(task)
    answers.append(answer)
    quesses.append(quess)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:54<00:00,  5.44s/it]


In [35]:
from pprint import pprint

In [36]:
pprint(list(zip(answers, quesses)))

[('1.6', '1.6'),
 ('24', '24'),
 ('32', '32'),
 ('256', '256'),
 ('53', '35'),
 ('3', '4'),
 ('27000; 64000', '1256'),
 ('2', '2'),
 ('0.1', '0.1'),
 ('1.2', '1.2')]


In [37]:
scoreing = sum(x == y for x, y in zip(answers, quesses)) / len(answers)
print(f"the score is {scoreing}")

the score is 0.7


# Try to sumbit dummy answer

In [42]:
def make_submit(answers: pd.Series, filename="data/my_submit.csv"):
    answer_df = answers.reset_index()
    answer_df.columns = ["problem_id", "answer"]
    answer_df.to_csv(filename, index=False)
    

In [38]:
test_df = pd.read_csv("data/test.csv", index_col=0)

In [43]:
answers = test_df.apply(lambda row: get_answer_2(row['problem_text']), axis=1)

In [44]:
make_submit(answers)